### In this file, we will define our model and parameters

Instructions: 

Finally, you should have at least one notebook where you define and train your notebook.

In [84]:
import pandas as pd
import numpy as np
import torch
from dgl.data import DGLDataset
from dgl.data.utils import save_graphs, load_graphs

### Model

In [85]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

from dgl.nn.pytorch import NNConv
from dgl.nn.pytorch import TAGConv

#### Ronan make your model here

In [86]:
# Model

#### RJ model

In [94]:
# Things to experiment with: SumPooling layer? Kind of like a conv, can sum up neighbors and find important structural properties of molecule
# from dgl.nn import SumPooling

# Also a good read here (https://arxiv.org/pdf/1710.10370.pdf) on Topology Adaptive Graph Convolutional layer might be useful.
# From my takeaways, we can use this "topology" to better track/weight the important parts of the structure
# (maybe the center of the molecule is important, or specific edge portions of the molecule) TAG can find this

# SAGEConv

# Ultimately, the NNConv layer, as used in the MPNN model, is the best one for this situation.
# We have come to this conclusion after trying various other convolution modules
# and finally reading up on this paper which relates closely to what we are doing (https://arxiv.org/pdf/1704.01212.pdf)
# (Quantum Chemistry computations)

# Note they used GatedGraphConv first and that worked okay but not as well as NNConv... we could look to backtrack to GGC
class Electron_MPNN(nn.Module):
    def __init__(self, node_in_feats, edge_in_feats, node_out_feats=64,
                 edge_hidden_feats=128, message_passing_steps=6):
        super(Electron_MPNN, self).__init__()

        # Projection
        self.fc1 = nn.Sequential(
            nn.Linear(node_in_feats, node_out_feats),
            nn.ReLU()
        )
    
        # Multi-iteration Convolution
        self.message_passing_steps = message_passing_steps
        edge_func = nn.Sequential(
            nn.Linear(edge_in_feats, edge_hidden_feats),
            nn.ReLU(),
            nn.Linear(edge_hidden_feats, node_out_feats * node_out_feats)
        )
        
        self.gnn_layer = NNConv(
            in_feats=node_out_feats,
            out_feats=node_out_feats,
            edge_func=edge_func,
            aggregator_type='sum'
        )
        
        # GRU used in MPNN to get rid of vanishing gradient on Conv layer
        # However, I think it is just added complexity to our model, so we will keep it here for reference but not use it
        self.gru = nn.GRU(node_out_feats, node_out_feats)
        
#         self.fc2 = nn.Linear(out_dim, out_dim)


    def forward(self, g, node_feats, edge_feats):
        node_feats = self.fc1(node_feats) # (V, node_out_feats)
        
        hidden_feats = node_feats.unsqueeze(0)           # (1, V, node_out_feats)

#         node_feats = self.gnn_layer(g, node_feats, edge_feats)
        
        for _ in range(self.message_passing_steps):
            node_feats = self.gnn_layer(g, node_feats, edge_feats)
            # GRU requires 3 dimensions
            node_feats, hidden_feats = self.gru(node_feats.unsqueeze(0), hidden_feats)
            node_feats = node_feats.squeeze(0)
            
#         node_feats = self.fc2(node_feats)
#         print(node_feats)
#         print("last_layer", node_feats)
        return node_feats

In [95]:
from dgl.nn.pytorch import Set2Set
class MPNNPredictor(nn.Module):
    def __init__(self,
                 node_in_feats,
                 edge_in_feats,
                 node_out_feats=64,
                 edge_hidden_feats=128,
                 n_tasks=1,
                 num_step_set2set=6,
                 num_layer_set2set=3,
                 message_passing_steps=6):
        super(MPNNPredictor, self).__init__()

        self.gnn = Electron_MPNN(node_in_feats=node_in_feats,
                                 node_out_feats=node_out_feats,
                                 edge_in_feats=edge_in_feats,
                                 edge_hidden_feats=edge_hidden_feats,
                                 message_passing_steps=message_passing_steps
                                )
                    
        self.readout = Set2Set(input_dim=node_out_feats,
                               n_iters=num_step_set2set,
                               n_layers=num_layer_set2set)
        
        self.predict = nn.Sequential(
            nn.Linear(2 * node_out_feats, node_out_feats),
            nn.ReLU(),
            nn.Linear(node_out_feats, n_tasks)
        )

    def forward(self, g, node_feats, edge_feats):
        node_feats = self.gnn(g, node_feats, edge_feats)
        graph_feats = self.readout(g, node_feats)
        return self.predict(graph_feats)

#### Load the graphs to get dimensions for our model

In [96]:
graphs, _ = load_graphs("./DataGraphs/data_F_graph.bin")

#### Construct the model

In [97]:
# All graphs in the list have the same scheme size, so pull the dimensions from the first
node_dim = graphs[0].ndata['atom_feats'].shape[1]
edge_dim = graphs[0].edata['bond_feats'].shape[1]
print("Dimensions:", node_dim, "(node),", edge_dim, "(edge)")

Dimensions: 11 (node), 5 (edge)


In [98]:
model = MPNNPredictor(node_dim, edge_dim)
#model = Electron_MPNN(node_dim, edge_dim)

In [99]:
sample_data = graphs[1]
edges = sample_data.edata['bond_feats'].float()
nodes = sample_data.ndata['atom_feats'].float()

In [100]:
model(sample_data, nodes, edges)

tensor([[0.1101]], grad_fn=<AddmmBackward>)

In [30]:
from os.path import exists
if exists('SavedModels/bestmae.txt'):
    with open('SavedModels/bestmae.txt', 'r') as f:
        txt = f.read()
        print(txt)
        print(float(txt))

11490.814183533093
11490.814183533093


In [21]:
best_mae = 123879.123
with open('SavedModels/bestmae.txt', 'w') as f:
    f.write(str(best_mae))